In [6]:
import os
import time

import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch.utils.data import DataLoader, Subset
from torch.utils.tensorboard import SummaryWriter
from torchvision import transforms
import torchvision.utils as tvutils

# import cv2
# import PIL
# import matplotlib.pyplot as plt

from my_utils.vis import *
from my_utils import models
from my_utils.utils import check_accuracy
from my_utils.data.dataset import make_dataset, MyDataset_actions, MyDataset_positions, raw_png_processor, raw_video_processor
from my_utils.data.loader import load_frames
from my_utils.data.preprocess import sub_mean, reduce

%matplotlib inline
plt.rcParams['figure.figsize'] = (10.0, 8.0)  # set default size of plots

# life save magic code
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
print(f"torch version: {torch.__version__}")
use_cuda = torch.cuda.is_available()
if use_cuda:
    GPU_nums = torch.cuda.device_count()
    GPU = torch.cuda.get_device_properties(0)
    print(f"There are {GPU_nums} GPUs in total.\nThe first GPU is: {GPU}")
    if '3060' in GPU.name:
        print(f"CUDA version: {torch.cuda_version}")
    else:
        print(f"CUDA version: {torch.version.cuda}")
device = torch.device(f"cuda:0" if use_cuda else "cpu")
print(f"Using {device} now!")

torch version: 1.10.2+cu113
There are 8 GPUs in total.
The first GPU is: _CudaDeviceProperties(name='Tesla V100-SXM2-32GB', major=7, minor=0, total_memory=32510MB, multi_processor_count=80)
CUDA version: 11.3
Using cuda:0 now!


In [8]:
dp = raw_png_processor(
    project_root='/mnt/cfs/wangyh/blender/blank_wall/',
    dataset_name='variety',
    # mode='unseen'
)

In [9]:
dp.check_data()

100%|█████████████████████████████████████████████████████| 800/800 [00:12<00:00, 66.25it/s]

{'Clap': 160, 'Crouch to Stand': 160, 'Dance': 160, 'Idle': 160, 'Jump': 160}
floor	 {'white_tiling': 200, 'grey_tiling': 200, 'wooden_floor_1': 200, 'wooden_floor_2': 200}
rot_num	 [200, 200, 200, 200]
pos_num	 [160, 160, 160, 160, 160]


In [12]:
dp.build_dataset(noise_factor=0)

5 classes in total: ['Clap', 'Crouch to Stand', 'Dance', 'Idle', 'Jump']


100%|█████████████████████████████████████████████████████| 800/800 [15:13<00:00,  1.14s/it]
